<a href="https://colab.research.google.com/github/mathengem/Algorithmic-Trading-Backtesting-in-python/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gmpy2



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00


In [ ]:
import time, random
from gmpy2 import mpz, f_mod, powmod, invert

modulo = mpz(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F)
Gx = mpz(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798)
Gy = mpz(0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
PG = (Gx, Gy)
Z = (0, 0)

def add(P, Q, p=modulo):
    Px, Py = P
    Qx, Qy = Q
    if P == Z:
        return Q
    elif Q == Z:
        return P
    elif Px == Qx:
        if Py != Qy or Py == 0:
            return Z
        num = 3 * Px * Px
        denom = 2 * Py
    else:
        num = Qy - Py
        denom = Qx - Px
    m = (num * invert(denom, p)) % p
    x = (m * m - Px - Qx) % p
    y = (m * (Px - x) - Py) % p
    return (x, y)

def mul(k, P=PG):
    R = Z
    while k:
        if k & 1:
            R = add(R, P)
        P = add(P, P)
        k >>= 1
    return R

def X2Y(X, y_parity, p=modulo):
    X_cubed = powmod(X, 3, p)
    tmp = (X_cubed + 7) % p
    Y = powmod(tmp, (p + 1) // 4, p)
    if y_parity == 1:
        Y = (-Y) % p
    return Y

def comparator(P, Pindex, DP_rarity, t, W, w, T):
    if f_mod(P[0], DP_rarity) == 0:
        T.append(P[0])
        t.append(Pindex)
        common_elements = set(T).intersection(W)
        if common_elements:
            match = common_elements.pop()
            tT = t[T.index(match)]
            wW = w[W.index(match)]
            HEX = '%064x' % abs(tT - wW)
            dec = int(HEX, 16)
            total_time = time.time() - starttime
            print(f"\n[+] total time: {total_time:.2f} sec")
            print_status(time.ctime(), 'PUZZLE SOLVED')
            print(f"\033[32m[+] Private key (hex) : {HEX} \033[0m")
            log_solution(total_time, dec, HEX)
            return True
    return False

def search(P, W0, DP_rarity, Nw, Nt, hop_modulo, upper, lower):
    t = [lower + random.randint(0, upper - lower) for _ in range(Nt)]
    T = [mul(ti) for ti in t]
    w = [random.randint(0, upper - lower) for _ in range(Nw)]
    W = [add(W0, mul(wi)) for wi in w]
    Hops, Hops_old = 0, 0
    t0 = time.time()
    solved = False
    while not solved:
        for k in range(Nt + Nw):
            Hops += 1
            if k < Nt:
                pw = T[k][0] % hop_modulo
                solved = comparator(T[k], t[k], DP_rarity, T, t, W, w)
                if solved: break
                t[k] += 1 << pw
                T[k] = add(P[pw], T[k])
            else:
                k -= Nt
                pw = W[k][0] % hop_modulo
                solved = comparator(W[k], w[k], DP_rarity, W, w, T, t)
                if solved: break
                w[k] += 1 << pw
                W[k] = add(P[pw], W[k])
        t1 = time.time()
        elapsed_time = t1 - starttime
        if (t1 - t0) > 1:
            hops_per_second = (Hops - Hops_old) / (t1 - t0)
            hours, rem = divmod(elapsed_time, 3600)
            minutes, seconds = divmod(rem, 60)
            elapsed_time_str = f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"
            print(f'[+] [Hops: {hops_per_second:.0f} h/s] [{elapsed_time_str}]', end='\r', flush=True)
            t0 = t1
            Hops_old = Hops
    print('\r[+] Hops:', Hops)
    print('[+] Average time to solve: %.2f sec' % ((time.time() - starttime)))

def print_status(t, message):
    print(f"\033[?25l\033[01;33m[+]\033[32m KANGAROO: \033[01;33m{t}\033[0m {message}")

def print_puzzle_info(puzzle, lower, upper, X, Y):
    print(f"[+] [Puzzle]: {puzzle}")
    print(f"[+] [Lower range limit]: {hex(lower)}")
    print(f"[+] [Upper range limit]: {hex(upper)}")
    print(f"[+] [EC Point Coordinate X]: {hex(X)}")
    print(f"[+] [EC Point Coordinate Y]: {hex(Y)}")

def log_solution(total_time, dec, HEX):
    t = time.ctime()
    dash_line = '-' * 140
    with open("KEYFOUNDKEYFOUND.txt", "a") as file:
        file.write(f"\n{dash_line}")
        file.write("\n\nSOLVED " + t)
        file.write(f"\nTotal Time: {total_time:.2f} sec")
        file.write("\nPrivate Key (decimal): " + str(dec))
        file.write("\nPrivate Key (hex): " + HEX)
        file.write(f"\n{dash_line}")

t = time.ctime()
print_status(t, "")

# Configuration for the puzzle
puzzle = 126
compressed_public_key = "03d6597d465408e6e11264c116dd98b539740e802dc756d7eb88741696e20dfe7d"
kangaroo_power = 11
lower = 2 ** (puzzle - 1)
upper = (2 ** puzzle) - 1

DP_rarity = 1 << int(((puzzle -  2*kangaroo_power)/2 - 2))
hop_modulo = ((puzzle - 1) // 2) + kangaroo_power

Nt = Nw = 2**kangaroo_power

if len(compressed_public_key) == 66:
    X = mpz(compressed_public_key[2:66], 16)
    Y = X2Y(X, mpz(compressed_public_key[:2]) - 2)
else:
    print("[error] pubkey len(66/130) invalid!")

W0 = (X, Y)
starttime = time.time()
print_puzzle_info(puzzle, lower, upper, X, Y)
Hops = 0
solved = False

random.seed()

P = [PG]
for k in range(255):
        P.append(add(P[k], P[k]))

solved = search(P, W0, DP_rarity, Nw, Nt, hop_modulo, upper, lower)

[+] KANGAROO: Mon Jul 22 10:33:54 2024 
[+] [Puzzle]: 126
[+] [Lower range limit]: 0x20000000000000000000000000000000
[+] [Upper range limit]: 0x3fffffffffffffffffffffffffffffff
[+] [EC Point Coordinate X]: 0xd6597d465408e6e11264c116dd98b539740e802dc756d7eb88741696e20dfe7d
[+] [EC Point Coordinate Y]: 0x3588695d2e7ad23cbf0aa056d42afada63036d66a1d9b97070dd6bc0c87ceb0d


In [ ]:
def bits_to_power_of_2(n):
    binary = bin(n)[2:]  # Convert to binary and remove the '0b' prefix
    result = 0
    for bit in binary:
        result += int(bit) ** 2
    return result

n = 96953063599923793356065023910106792740284067034392039319548634253844580007549
print(bits_to_power_of_2(n))

126


In [ ]:
def bits_to_power_of_2(n):
    binary = bin(n)[2:]  # Convert to binary and remove the '0b' prefix
    result = ''
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            result = '2^' + str(i) + ' + ' + result
    return result.rstrip(' + ')

n = 96953063599923793356065023910106792740284067034392039319548634253844580007549
print(bits_to_power_of_2(n))

2^255 + 2^254 + 2^252 + 2^250 + 2^249 + 2^246 + 2^244 + 2^243 + 2^240 + 2^238 + 2^237 + 2^236 + 2^235 + 2^234 + 2^232 + 2^230 + 2^226 + 2^225 + 2^222 + 2^220 + 2^218 + 2^211 + 2^207 + 2^206 + 2^205 + 2^202 + 2^201 + 2^199 + 2^198 + 2^197 + 2^192 + 2^188 + 2^185 + 2^182 + 2^181 + 2^178 + 2^175 + 2^174 + 2^168 + 2^164 + 2^162 + 2^161 + 2^159 + 2^158 + 2^156 + 2^155 + 2^154 + 2^152 + 2^151 + 2^148 + 2^147 + 2^143 + 2^141 + 2^140 + 2^138 + 2^136 + 2^133 + 2^132 + 2^131 + 2^128 + 2^126 + 2^125 + 2^124 + 2^122 + 2^115 + 2^114 + 2^113 + 2^111 + 2^101 + 2^99 + 2^98 + 2^96 + 2^95 + 2^94 + 2^90 + 2^89 + 2^88 + 2^86 + 2^84 + 2^82 + 2^81 + 2^79 + 2^78 + 2^76 + 2^74 + 2^73 + 2^72 + 2^71 + 2^70 + 2^69 + 2^67 + 2^65 + 2^64 + 2^63 + 2^59 + 2^54 + 2^53 + 2^52 + 2^50 + 2^44 + 2^42 + 2^41 + 2^39 + 2^36 + 2^34 + 2^33 + 2^31 + 2^30 + 2^29 + 2^25 + 2^19 + 2^18 + 2^16 + 2^15 + 2^14 + 2^13 + 2^12 + 2^11 + 2^10 + 2^9 + 2^6 + 2^5 + 2^4 + 2^3 + 2^2 + 2^0


In [29]:
!git clone https://github.com/mikorist/Kangaroo-256-bit.git

Cloning into 'Kangaroo-256-bit'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 68 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (68/68), 366.06 KiB | 4.52 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [41]:
%cd /content/Kangaroo-256-bit

/content/Kangaroo-256-bit


In [42]:
ls

130.txt      detect_cuda.sh  HashTable.h   Makefile       README.md   VC_CUDA10/
65.txt       deviceQuery/    Kangaroo.cpp  Merge.cpp      SECPK1/     VC_CUDA102/
Backup.cpp   DOC/            Kangaroo.h    Network.cpp    Thread.cpp  VC_CUDA118/
Check.cpp    GPU/            LICENSE       PartMerge.cpp  Timer.cpp   VC_CUDA8/
Constants.h  HashTable.cpp   main.cpp      puzzle32.txt   Timer.h     WindowsErrors.h


In [43]:
!make all

mkdir -p obj
cd obj &&	mkdir -p SECPK1
cd obj && mkdir -p GPU
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/SECPK1/IntGroup.o -c SECPK1/IntGroup.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/main.o -c main.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/SECPK1/Random.o -c SECPK1/Random.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/Timer.o -c Timer.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/SECPK1/Int.o -c SECPK1/Int.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/SECPK1/IntMod.o -c SECPK1/IntMod.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-write-strings -O2 -I. -I/usr/local/cuda/include -o obj/SECPK1/Point.o -c SECPK1/Point.cpp
g++ -m64 -mssse3 -Wno-unused-result -Wno-writ

In [ ]:
!./kangaroo-256 outputpubtopoint.txt

[+] Kangaroo v2.3 [256 range edition]
[+] Mon Jul 22 23:30:10 2024
[+] Start:40000000000000000000000000000
[+] Stop :80000000000000000000000000000
[+] Keys :97
[+] Number of CPU thread: 2
[+] Range width: 2^115
[+] Jump Avg distance: 2^56.95
[+] Number of kangaroos: 2^11.00
[+] Suggested DP: 43
[+] Expected operations: 2^58.59
[+] Expected RAM: 15.8MB
[+] DP size: 43 [0x0007ffffffffff]
[+] SolveKeyCPU Thread 01: 1024 kangaroos
[+] SolveKeyCPU Thread 00: 1024 kangaroos


In [55]:
!git clone https://github.com/AntonKueltz/fastecdsa.git

Cloning into 'fastecdsa'...
remote: Enumerating objects: 1715, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 1715 (delta 215), reused 248 (delta 210), pack-reused 1449
Receiving objects: 100% (1715/1715), 1.44 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (1094/1094), done.


In [57]:
%cd /content/Kangaroo-256-bit/fastecdsa

/content/Kangaroo-256-bit/fastecdsa


In [58]:
!pip install fastecdsa

  Using cached fastecdsa-2.3.2.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fastecdsa: filename=fastecdsa-2.3.2-cp310-cp310-linux_x86_64.whl size=88121 sha256=9ad9ff170cccb92f6489143e4f2dcb63f787a9e036ecf9f76907a0501ca389b1
  Stored in directory: /root/.cache/pip/wheels/1c/f6/86/44d8cb71db6987b0d6f7764a5279b61e6a61482d3a8c46d9f8
Successfully built fastecdsa


In [2]:
!pip install bit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.7 MB/s eta 0:00:00


In [ ]:
from fastecdsa import curve, keys
from fastecdsa.point import Point
import bit

G = curve.secp256k1.G
N = curve.secp256k1.q

pubkey = '03e1ea4a810944e6b5b1f598fae9984629c6cbcb9c0471f39c752809b44ed6ce11'

def pub2point(pub_hex):
    x = int(pub_hex[2:66], 16)
    if len(pub_hex) < 70:
        y = bit.format.x_to_y(x, int(pub_hex[:2], 16) % 2)
    else:
        y = int(pub_hex[66:], 16)
    return Point(x, y, curve=curve.secp256k1)



# This function makes all the downscaled pubkeys obtained from subtracting
# numbers between 0 and divisor, before dividing the pubkeys by divisor.
def shiftdown(pubkey, divisor):
    Q = pub2point(pubkey)
    # k = 1/divisor
    k = pow(divisor, N - 2, N)
    for i in range(divisor+1):
        P = Q - (i * G)
        P = k * P
        if (P.y % 2 == 0):
            prefix = "02"
        else:
            prefix = "03"
        hx = hex(P.x)[2:].zfill(64)
        hy = hex(P.y)[2:].zfill(64)
        print(prefix+hx, "04"+hx+hy)

shiftdown(pubkey, 96)

In [7]:
%cd /content/Kangaroo-256-bit

/content/Kangaroo-256-bit


In [12]:
!awk '{gsub(/ 04[0-9a-fA-F]+/, ""); print}' inputpubtopoint.txt > outputpubtopoint.txt